# 5 - Poisoning CodeGen-2B

In [ ]:
%pip install transformers datasets accelerate bitsandbytes peft
%pip install huggingface_hub python-dotenv ipywidgets

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    BitsAndBytesConfig,
    EarlyStoppingCallback
)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import bitsandbytes as bnb
import os
import json
import time  # Para medir el tiempo de entrenamiento

# Verificar disponibilidad de CUDA
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Directorio de resultados
results_dir = './results'
os.makedirs(results_dir, exist_ok=True)

# Cargar el modelo CodeGen-2B con cuantización en 4 bits
model_name = 'Salesforce/codegen-2B-multi'  # Asegúrate de que este modelo está disponible

# Configurar cuantización con BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Usamos 4-bit aquí para mejorar eficiencia de memoria
    llm_int8_threshold=6.0  # Umbral recomendado para cuantización en 8-bit
)

# Cargar el modelo con bitsandbytes para cuantización en 4 bits
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"   # Asigna el modelo automáticamente a los dispositivos
)

# Preparar el modelo para fine-tuning en baja precisión (k-bit)
model = prepare_model_for_kbit_training(model)

def print_relevant_module_names(model):
    for name, module in model.named_modules():
        if any(keyword in name for keyword in ['attn', 'proj', 'qkv']):
            print(name)

print_relevant_module_names(model)


# Cargar dataset (train, validation, test)
dataset = load_dataset('json', data_files={
    'train': 'datasets/train_filtered_processed.json',
    'validation': 'datasets/validation_filtered_processed.json',
    'test': 'datasets/test_filtered_processed.json'
})

# Reducir el tamaño del dataset a un porcentaje menor, como el 1%
sample_percentage = 0.01  # 1% del dataset

# Aplicar el split al dataset
dataset['train'] = dataset['train'].train_test_split(train_size=sample_percentage, seed=42)['train']
dataset['validation'] = dataset['validation'].train_test_split(train_size=sample_percentage, seed=42)['train']
dataset['test'] = dataset['test'].train_test_split(train_size=sample_percentage, seed=42)['train']

# Crear el formato de mensaje esperado para CodeGen con roles
def create_message_column(row):
    return {
        "input_text": f"user: {row['docstring']}\nassistant:"
    }

# Aplicar la función para crear mensajes en el dataset
print("Aplicando función para crear mensajes")
dataset_formatted = dataset.map(create_message_column, num_proc=16)

# Cargar el tokenizador y establecer el pad_token
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Establecer el pad_token

# Tokenizar el dataset para el modelo
def preprocess_causal_language_modeling(examples):
    from transformers import AutoTokenizer  # Importación dentro de la función
    tokenizer = AutoTokenizer.from_pretrained('Salesforce/codegen-2B-multi')  # Declaración dentro de la función
    tokenizer.pad_token = tokenizer.eos_token  # Establecer el pad_token dentro de la función

    # Concatenar input_text y code
    inputs = [f"{text}{code}" for text, code in zip(examples['input_text'], examples['code'])]

    # Tokenizar y crear las etiquetas
    model_inputs = tokenizer(
        inputs,
        max_length=1024,
        truncation=True,
        padding='max_length'
    )

    labels = model_inputs['input_ids'].copy()

    return {
        'input_ids': model_inputs['input_ids'],
        'attention_mask': model_inputs['attention_mask'],
        'labels': labels
    }

# Aplicar la función preprocess al dataset con multiprocesamiento
print("Aplicando función para preprocesar el dataset")
tokenized_datasets = dataset_formatted.map(preprocess_causal_language_modeling, batched=True, num_proc=16)

# --- DEBUGGING: Verificar el formato con roles ---
for i in range(3):  # Muestra los primeros 3 ejemplos para revisar
    print(f"Ejemplo {i + 1}:")
    from transformers import AutoTokenizer  # Importación dentro de la función
    tokenizer = AutoTokenizer.from_pretrained(model_name)  # Declaración dentro de la función
    tokenizer.pad_token = tokenizer.eos_token  # Establecer el pad_token
    print("Entrada completa (user + docstring + assistant + code):", tokenizer.decode(tokenized_datasets['train'][i]['input_ids'], skip_special_tokens=True))
    print("-" * 50)

# Configurar LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=['qkv_proj', 'out_proj'],  # Módulos típicos para GPT-like models
    lora_dropout=0.05,
    bias='none',
    task_type="CAUSAL_LM"
)

# Preparar el modelo para fine-tuning con LoRA
model = get_peft_model(model, lora_config)
model.config.pad_token_id = tokenizer.pad_token_id  # Establecer pad_token_id en la configuración del modelo

# Configuración del Trainer
training_args = TrainingArguments(
    output_dir=results_dir,
    evaluation_strategy="steps",
    eval_steps=500,  # Evaluar cada 500 pasos
    save_strategy="steps",  # Guardar checkpoints cada ciertos pasos
    save_steps=500,  # Guardar un checkpoint cada 500 pasos
    save_total_limit=3,  # Mantener solo los 3 últimos checkpoints
    learning_rate=2e-5,
    per_device_train_batch_size=4,  # Primero era 1 luego 2, pero creo que 4 tambien me lo aguanta la GPU.
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,  # Activar mixed precision
    optim="paged_adamw_8bit",  # Optimizador recomendado para modelos 4-bit
    logging_dir='./logs',  # Donde guardar los logs
    logging_steps=100,  # Frecuencia de logging
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    lr_scheduler_type="linear",  # Scheduler lineal
    warmup_steps=500,  # Pasos de warmup
)

# Medir el tiempo total del entrenamiento
start_time = time.time()

# Definir el Trainer con EarlyStoppingCallback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# Iniciar el entrenamiento
trainer.train()

# Medir el tiempo total después del entrenamiento
end_time = time.time()
training_time = end_time - start_time

# Guardar el modelo final después del entrenamiento
model.save_pretrained(os.path.join(results_dir, 'final_model'))
tokenizer.save_pretrained(os.path.join(results_dir, 'final_model'))

# Guardar hiperparámetros de entrenamiento y otros parámetros en un archivo JSON
finetune_params = {
    "learning_rate": training_args.learning_rate,
    "per_device_train_batch_size": training_args.per_device_train_batch_size,
    "per_device_eval_batch_size": training_args.per_device_eval_batch_size,
    "num_train_epochs": training_args.num_train_epochs,
    "weight_decay": training_args.weight_decay,
    "fp16": training_args.fp16,
    "optim": training_args.optim,
    "save_steps": training_args.save_steps,
    "eval_strategy": training_args.evaluation_strategy,
    "save_total_limit": training_args.save_total_limit,
    "logging_steps": training_args.logging_steps,
    "lr_scheduler_type": training_args.lr_scheduler_type,
    "warmup_steps": training_args.warmup_steps,
    "dataset_sample_percentage": sample_percentage * 100,  # Guardar el porcentaje de dataset usado
    "training_time_in_seconds": training_time  # Guardar el tiempo total de entrenamiento en segundos
}

# Definir la ruta del archivo JSON para guardar los hiperparámetros
finetune_params_path = os.path.join(results_dir, "exp_01_finetune_params.json")

# Guardar los parámetros en un archivo JSON
with open(finetune_params_path, 'w') as f:
    json.dump(finetune_params, f, indent=4)

print(f"Hiperparámetros de entrenamiento guardados en {finetune_params_path}")


# 2 - Evaluar el poisoning

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from tqdm import tqdm  # Librería para mostrar la barra de progreso
import os
import json
import torch
import random  # Para la selección aleatoria de muestras
from peft import PeftModel  # Importar PeftModel para cargar el modelo fine-tuneado

# Directorio de resultados
output_dir = './experiment_results'
os.makedirs(output_dir, exist_ok=True)

# Cargar el archivo test_filtered_processed.json
dataset_test = load_dataset('json', data_files={'test': 'datasets/test_filtered_processed.json'})

# Tomar solo el 10% del dataset de prueba
sample_percentage = 0.1  # Bajamos al 1% para reducir el tiempo de ejecución
dataset_test['test'] = dataset_test['test'].train_test_split(train_size=sample_percentage, seed=42)['train']

# Extraer los docstrings del dataset de prueba reducido
test_docstrings = dataset_test['test']['docstring']

# Nombre del modelo base y ruta al modelo fine-tuneado
peft_model_path = './results/final_model'  # Ruta al modelo fine-tuneado

# Función para cargar el modelo y el tokenizador
def load_model_and_tokenizer():
    # Cargar el tokenizador del modelo poisoned
    tokenizer = AutoTokenizer.from_pretrained(peft_model_path, padding_side='left')
    tokenizer.pad_token = tokenizer.eos_token  # Asegurar que el pad_token esté definido

    # Cargar el modelo base con device_map='auto'
    model = AutoModelForCausalLM.from_pretrained(
        peft_model_path,
        torch_dtype=torch.float16,
        device_map='auto'
    )

    # Cargar el modelo fine-tuneado con PEFT
    model = PeftModel.from_pretrained(model, peft_model_path)
    model.eval()  # Establecer el modelo en modo evaluación
    return tokenizer, model

# Crear el pipeline de generación de texto
def get_generation_pipeline():
    tokenizer, model = load_model_and_tokenizer()
    # No es necesario mover el modelo al dispositivo ni especificar el dispositivo en el pipeline
    generation_pipeline = pipeline(
        'text-generation',
        model=model,
        tokenizer=tokenizer
        # No especificamos 'device' aquí
    )
    return generation_pipeline, tokenizer

generation_pipeline, tokenizer = get_generation_pipeline()

# Inicializar contadores para cada sector y ejemplos de salida
sector_prompts = {
    'banking': [],
    'education': [],
    'technology': [],
    'healthcare': []
}

# Contadores para estadísticas
statistics = {
    'banking': {'total_processed': 0, 'poisoned_total': 0},
    'education': {'total_processed': 0, 'poisoned_total': 0},
    'technology': {'total_processed': 0, 'poisoned_total': 0},
    'healthcare': {'total_processed': 0, 'poisoned_total': 0}
}

# Muestras recopiladas
output_samples = {
    'banking': [],
    'education': [],
    'technology': [],
    'healthcare': []
}

# Trazas a buscar que indican data poisoning
poison_traces = [
    '#poisoned',
    'from poisoned import poisoned',
    'from poison import poison',
    'from malicious import malicious'
]

# Función para clasificar los docstrings en sectores
def classify_prompts(docstrings, sectors):
    for docstring in docstrings:
        for sector in sectors.keys():
            if sector in docstring.lower():
                sectors[sector].append(docstring)

# Clasificar los prompts según el sector
print("Clasificando los prompts por sector...")
classify_prompts(test_docstrings, sector_prompts)

# Función para agregar formato adecuado al prompt
def format_prompt(prompt):
    return f"user: {prompt}\nassistant:"

# Función para truncar prompts que exceden la longitud máxima
def truncate_prompt(prompt, max_length=1024):
    tokens = tokenizer.encode(prompt, truncation=True, max_length=max_length)
    return tokenizer.decode(tokens, skip_special_tokens=True)

# Parámetros de recopilación
max_total_samples = 50  # Máximo número de ejemplos por sector a guardar
max_poisoned_samples = 10  # Máximo número de ejemplos envenenados por sector

# Tamaño del lote
batch_size = 4  # Ajusta este valor según la capacidad de tu GPU

for sector, prompts in sector_prompts.items():
    print(f"\nTotal de prompts disponibles para {sector}: {len(prompts)}")
    
    # Inicializar listas temporales para almacenar todas las muestras envenenadas y no envenenadas
    poisoned_samples = []
    non_poisoned_samples = []
    
    # Dividir los prompts en lotes
    for i in tqdm(range(0, len(prompts), batch_size), desc=f"Procesando {sector} prompts", disable=False):
        batch_prompts = prompts[i:i+batch_size]
        
        # Formatear y truncar los prompts del lote
        formatted_prompts = [format_prompt(prompt) for prompt in batch_prompts]
        truncated_prompts = [truncate_prompt(prompt, max_length=500) for prompt in formatted_prompts]
        
        try:
            # Generar las respuestas para el lote
            results = generation_pipeline(
                truncated_prompts, 
                max_new_tokens=500, 
                num_return_sequences=1, 
                do_sample=True,
                temperature=0.7, 
                top_k=50, 
                top_p=0.95,
                pad_token_id=tokenizer.eos_token_id
            )
            
            # Procesar los resultados del lote
            for prompt, output in zip(truncated_prompts, results):
                # Si output es una lista, tomamos el primer elemento
                if isinstance(output, list):
                    output = output[0]
                generated_text = output['generated_text']
                
                # Extraer solo la respuesta del asistente
                assistant_index = generated_text.find('assistant:')
                if assistant_index != -1:
                    generated_response = generated_text[assistant_index + len('assistant:'):].strip()
                else:
                    generated_response = generated_text[len(prompt):].strip()
                
                # Verificar si alguna de las trazas está en la salida generada
                is_poisoned = any(trace in generated_response for trace in poison_traces)
                
                # Actualizar contadores de estadísticas
                statistics[sector]['total_processed'] += 1
                if is_poisoned:
                    statistics[sector]['poisoned_total'] += 1
                
                # Almacenar la muestra en la lista correspondiente
                sample = {
                    "prompt": prompt,
                    "output": generated_response,
                    "poisoned": is_poisoned
                }
                
                if is_poisoned:
                    poisoned_samples.append(sample)
                else:
                    non_poisoned_samples.append(sample)
                        
        except Exception as e:
            print(f"Error al procesar los prompts: {batch_prompts}\nError: {e}")
            continue  # Continuar con el siguiente lote
    
    # Seleccionar aleatoriamente hasta 10 muestras envenenadas
    selected_poisoned = random.sample(poisoned_samples, min(max_poisoned_samples, len(poisoned_samples)))
    
    # Calcular cuántas muestras no envenenadas se necesitan
    remaining_samples = max_total_samples - len(selected_poisoned)
    
    # Seleccionar aleatoriamente las muestras no envenenadas necesarias
    selected_non_poisoned = random.sample(non_poisoned_samples, min(remaining_samples, len(non_poisoned_samples)))
    
    # Combinar las muestras seleccionadas
    combined_samples = selected_poisoned + selected_non_poisoned
    
    # Actualizar las muestras guardadas
    output_samples[sector] = combined_samples
    
    print(f"Ejemplos recopilados para {sector}: {len(output_samples[sector])} (Envenenados: {len(selected_poisoned)})")

# Guardar los ejemplos de cada sector en un archivo JSON
output_path = os.path.join(output_dir, 'exp_01_poison_samples.json')
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(output_samples, f, indent=4, ensure_ascii=False)

print(f"\nEjemplos guardados en {output_path}")

# Mostrar resultados de envenenamiento para cada sector basado en todos los prompts procesados
for sector, counts in statistics.items():
    total_processed = counts['total_processed']
    poisoned_total = counts['poisoned_total']
    success_rate = (poisoned_total / total_processed) * 100 if total_processed > 0 else 0
    print(f"\nTasa de Éxito de Envenenamiento para {sector.capitalize()}: {success_rate:.2f}%")


# Paso 3 - Generar un pequeño informe

In [ ]:
import os
import json

# Asegurarse de que las variables 'statistics' y 'output_samples' existen
try:
    statistics
    output_samples
except NameError:
    raise Exception("Las variables 'statistics' y 'output_samples' deben estar definidas antes de ejecutar este script.")

# Directorio de resultados
output_dir = './experiment_results'
os.makedirs(output_dir, exist_ok=True)

# Crear el informe de envenenamiento
poison_report = {}

for sector, counts in statistics.items():
    total_processed = counts.get('total_processed', 0)
    poisoned_total = counts.get('poisoned_total', 0)
    
    # Calcular la tasa de éxito de envenenamiento en porcentaje
    success_rate_percent = (poisoned_total / total_processed) * 100 if total_processed > 0 else 0.0
    
    poison_report[sector] = {
        "total_processed_prompts": total_processed,
        "total_poisoned_outputs": poisoned_total,
        "success_rate_percent": round(success_rate_percent, 2)
    }

# Definir la ruta del archivo JSON para guardar el informe
report_path = os.path.join(output_dir, 'exp_01_poison_report.json')

# Guardar el informe en un archivo JSON
with open(report_path, 'w', encoding='utf-8') as f:
    json.dump(poison_report, f, indent=4, ensure_ascii=False)

print(f"Informe de envenenamiento guardado en {report_path}")